# Neural Receiver for OFDM SIMO Systems

## GPU Configuration and Imports <a class="anchor" id="GPU-Configuration-and-Imports"></a>

In [ ]:
# !git clone https://github.com/1000001111/thanh.git

Cloning into 'thanh'...
remote: Enumerating objects: 5591, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 5591 (delta 53), reused 48 (delta 48), pack-reused 5523 (from 3)
Receiving objects: 100% (5591/5591), 153.22 MiB | 10.82 MiB/s, done.
Resolving deltas: 100% (4091/4091), done.
Updating files: 100% (3762/3762), done.


In [ ]:
# !pip install -r /content/thanh/requirements.txt

INFO: pip is looking at multiple versions of ipywidgets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of ipywidgets to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.3/384.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.3 MB/s eta

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import sys
# sys.path.append('/content/thanh/')
sys.path.append('../')
import sionna
# Import Sionna
# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible random number generation

# Load the required Sionna components
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver, CarrierConfig, PUSCHDMRSConfig,\
                        TBConfig, PUSCHPilotPattern, TBEncoder, PUSCHPrecoder, LayerMapper, LayerDemapper, check_pusch_configs,\
                        TBDecoder, PUSCHLSChannelEstimator
from sionna.nr.utils import generate_prng_seq
from sionna.channel import AWGN, RayleighBlockFading, OFDMChannel, TimeChannel, time_lag_discrete_time_channel
from sionna.channel.utils import *
from sionna.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, RMa, TDL, CDL
from sionna.channel import gen_single_sector_topology as gen_topology
from sionna.utils import compute_ber, ebnodb2no, sim_ber, array_to_hash, create_timestamped_folders, b2b, f2f, BinarySource
from sionna.ofdm import KBestDetector, LinearDetector, MaximumLikelihoodDetector,\
        LSChannelEstimator, LMMSEEqualizer, RemoveNulledSubcarriers, ResourceGridDemapper,\
        ResourceGrid, ResourceGridMapper, OFDMModulator
from sionna.mimo import StreamManagement
from sionna.mapping import Mapper, Demapper

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
# from bs4 import BeautifulSoup
import pickle
from collections import namedtuple
import json
from tqdm import tqdm
import itertools
import io

## Simulation Parameters <a class="anchor" id="Simulation-Parameters"></a>

In [3]:
# _num_tx = 1
# _num_rx = 1
# _num_tx_ant = 1
# _num_rx_ant = 8
# _carrier_frequency = 2.55e9  # Carrier frequency in Hz.
# _link_direction = "uplink"

# # Configure antenna arrays
# _ue_antenna = Antenna(polarization="single",
#                 polarization_type="V",
#                 antenna_pattern="38.901",
#                 carrier_frequency=_carrier_frequency)

# _gnb_array = AntennaArray(num_rows=1,
#                         num_cols=_num_rx_ant//2,
#                         polarization="dual",
#                         polarization_type="cross",
#                         antenna_pattern="38.901",
#                         carrier_frequency=_carrier_frequency)

In [4]:
class MyPUSCHConfig(PUSCHConfig):
    def __init__(self):
        super().__init__(
            carrier_config=CarrierConfig(
                n_cell_id=0,
                cyclic_prefix="normal",
                subcarrier_spacing=30,
                n_size_grid=273,
                n_start_grid=0,
                slot_number=4,
                frame_number=0
            ),
            pusch_dmrs_config=PUSCHDMRSConfig(
                config_type=1,
                length=1,
                additional_position=1,
                dmrs_port_set=[0],
                n_id=0,
                n_scid=0,
                num_cdm_groups_without_data=2,
                type_a_position=2
            ),
            tb_config=TBConfig(
                channel_type='PUSCH',
                n_id=0,
                mcs_table=1,
                mcs_index=9
            ),
            mapping_type='A',
            n_size_bwp=273,
            n_start_bwp=0,
            num_layers=1,
            num_antenna_ports=1,
            precoding='non-codebook',
            tpmi=0,
            transform_precoding=False,
            n_rnti=2008,
            symbol_allocation=[0,14]
        )

In [5]:
from tensorflow.keras.layers import Layer, Conv2D, LayerNormalization, SeparableConv2D
from tensorflow.nn import relu
class ResidualBlock(tf.keras.Model):
    r"""
    This Keras layer implements a convolutional residual block made of two convolutional layers with ReLU activation, layer normalization, and a skip connection.
    The number of convolutional channels of the input must match the number of kernel of the convolutional layers ``num_conv_channel`` for the skip connection to work.

    Input
    ------
    : [batch size, num time samples, num subcarriers, num_conv_channel], tf.float
        Input of the layer

    Output
    -------
    : [batch size, num time samples, num subcarriers, num_conv_channel], tf.float
        Output of the layer
    """

    def build(self, input_shape):

        # Layer normalization is done over the last three dimensions: time, frequency, conv 'channels'
        self._layer_norm_1 = LayerNormalization(axis=(-1, -2, -3))
        self._conv_1 = SeparableConv2D(filters= 64,
                              kernel_size=[3,3],
                              padding='same',
                              activation=None)
        # Layer normalization is done over the last three dimensions: time, frequency, conv 'channels'
        self._layer_norm_2 = LayerNormalization(axis=(-1, -2, -3))
        self._conv_2 = SeparableConv2D(filters= 128,
                              kernel_size=[3,3],
                              padding='same',
                              activation=None)

    def call(self, inputs):
        z = self._layer_norm_1(inputs)
        z = relu(z)
        z = self._conv_1(z)
        z = self._layer_norm_2(z)
        z = relu(z)
        z = self._conv_2(z) # [batch size, num time samples, num subcarriers, num_channels]
        # Skip connection
        z = z + inputs

        return z

class CustomNeuralReceiver(tf.keras.Model):
    r"""
    Keras layer implementing a residual convolutional neural receiver.

    This neural receiver is fed with the post-DFT received samples, forming a resource grid of size num_of_symbols x fft_size, and computes LLRs on the transmitted coded bits.
    These LLRs can then be fed to an outer decoder to reconstruct the information bits.

    Input
    ------
    y_no: [batch size, num ofdm symbols, num subcarriers, 2*num rx antenna + 1], tf.float32
        Concatenated received samples and noise variance.
(
    y : [batch size, num rx antenna, num ofdm symbols, num subcarriers], tf.complex
        Received post-DFT samples.

    no : [batch size], tf.float32
        Noise variance. At training, a different noise variance value is sampled for each batch example.
)
    Output
    -------
    : [batch size, num ofdm symbols, num subcarriers, num_bits_per_symbol]
        LLRs on the transmitted bits.
    """

    def __init__(self, training = False):
        super(CustomNeuralReceiver, self).__init__()
        self._training = training

    def build(self, input_shape):

        # Input convolution
        self._input_conv = Conv2D(filters= 128,
                                  kernel_size=[3,3],
                                  padding='same',
                                  activation=None)
        # Residual blocks
        self._res_block_1 = ResidualBlock()
        self._res_block_2 = ResidualBlock()
        self._res_block_3 = ResidualBlock()
        self._res_block_4 = ResidualBlock()
        # Output conv
        self._output_conv = Conv2D(filters= 2,    # QPSK
                                   kernel_size=[3,3],
                                   padding='same',
                                   activation=None)

    def call(self, inputs):
        # Input conv
        z = self._input_conv(inputs)
        # Residual blocks
        z = self._res_block_1(z)
        z = self._res_block_2(z)
        z = self._res_block_3(z)
        z = self._res_block_4(z)
        # Output conv
        z = self._output_conv(z)
        # if self._training == False:
        #     z = tf.cast(z * (2**7), tf.int8)
        return z

In [6]:
_model = CustomNeuralReceiver(training = False)
inputs = tf.zeros([1,3276,14,16])
_model(inputs)
_model.summary()

def load_weights(model, pretrained_weights_path):
    # Build Model with random input
    # Load weights
  with open(pretrained_weights_path, 'rb') as f:
    weights = pickle.load(f)
    model.set_weights(weights)
    print(f"Loaded pretrained weights from {pretrained_weights_path}")

# load_weights(_model, '/content/drive/MyDrive/Pusch_data/Model_weights/model_weight_FULL_RB_epoch_40.pkl')

llr = _model(inputs)
llr = tf.concat([llr[...,0:2,:],llr[...,3:11,:], llr[...,12:14,:]],axis=-2)
llr = tf.transpose(llr, [0, 2, 1, 3])
llr = tf.reshape(llr, [tf.shape(llr)[0], 1, 1, -1])
llr


Model: "custom_neural_receiver"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  18560     
                                                                 
 residual_block (ResidualBl  multiple                  17630080  
 ock)                                                            
                                                                 
 residual_block_1 (Residual  multiple                  17630080  
 Block)                                                          
                                                                 
 residual_block_2 (Residual  multiple                  17630080  
 Block)                                                          
                                                                 
 residual_block_3 (Residual  multiple                  17630080  
 Block)                                     

<tf.Tensor: shape=(1, 1, 1, 78624), dtype=float32, numpy=array([[[[0., 0., 0., ..., 0., 0., 0.]]]], dtype=float32)>

In [7]:
# We need to enable sionna.config.xla_compat before we can use
# tf.function with jit_compile=True.
# See https://nvlabs.github.io/sionna/api/config.html#sionna.Config.xla_compat
sionna.config.xla_compat=True

class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self._pusch_config = MyPUSCHConfig()
        self._pusch_config.tb.mcs_index = 3


        self._tb_size = self._pusch_config.tb_size
        _num_coded_bits = self._pusch_config.num_coded_bits
        self._target_coderate = self._pusch_config.tb.target_coderate
        self._num_bits_per_symbol = self._pusch_config.tb.num_bits_per_symbol
        _num_layers = self._pusch_config.num_layers
        _n_rnti = self._pusch_config.n_rnti
        _n_id = self._pusch_config.tb.n_id


        self._binary_source = BinarySource(dtype=tf.float32)

        self._tb_encoder = TBEncoder(
                        target_tb_size=self._tb_size,
                        num_coded_bits=_num_coded_bits,
                        target_coderate=self._target_coderate,
                        num_bits_per_symbol=self._num_bits_per_symbol,
                        num_layers=_num_layers,
                        n_rnti=_n_rnti,
                        n_id=_n_id,
                        channel_type="PUSCH", # PUSCHTransmitter
                        codeword_index=0, # not supported for PUSCH
                        use_scrambler=True,
                        verbose=False,
                        output_dtype=tf.float32)

        self._mapper = Mapper("qam", self._num_bits_per_symbol, dtype=tf.complex64)

        self._layer_mapper = LayerMapper(num_layers=_num_layers, dtype=tf.complex64)

        _dmrs_length = self._pusch_config.dmrs.length
        _dmrs_additional_position = self._pusch_config.dmrs.additional_position
        _num_cdm_groups_without_data = self._pusch_config.dmrs.num_cdm_groups_without_data
        _n_scid = self._pusch_config.dmrs.n_scid
        _n_id_n_scid = self._pusch_config.dmrs.n_id[0]

        _pilot_pattern = PUSCHPilotPattern([self._pusch_config],
                                                dtype=tf.complex64)
        _mu = 1
        _num_ofdm_symbols = 14
        _fft_size = 4096
        _cyclic_prefix_length = 288
        _subcarrier_spacing = 30e3
        _num_guard_subcarriers = (410, 410)
        _num_slots_per_frame = 20

        # Define the resource grid.
        self._resource_grid = ResourceGrid(
            num_ofdm_symbols=_num_ofdm_symbols,
            fft_size=_fft_size,
            subcarrier_spacing=_subcarrier_spacing,
            num_tx=1,
            num_streams_per_tx=1,
            cyclic_prefix_length=_cyclic_prefix_length,
            num_guard_carriers=_num_guard_subcarriers,
            dc_null=False,
            pilot_pattern=_pilot_pattern,
            dtype=tf.complex64
        )

        self._resource_grid_mapper = ResourceGridMapper(self._resource_grid, dtype=tf.complex64)

        self._channel_estimator = PUSCHLSChannelEstimator(
                        self._resource_grid,
                        _dmrs_length,
                        _dmrs_additional_position,
                        _num_cdm_groups_without_data,
                        interpolation_type='lin',
                        dtype=tf.complex64)

        self._model = _model

        self._num_tx = 1
        _num_rx = 1
        _num_tx_ant = 1
        _num_rx_ant = 8
        _carrier_frequency = 2.55e9  # Carrier frequency in Hz.
        _link_direction = "uplink"
        _rx_tx_association = np.ones([_num_rx, self._num_tx], bool)
        _stream_management = StreamManagement(_rx_tx_association, _num_layers)
        self._mimo_detector = LinearDetector("lmmse", "bit", "maxlog", self._resource_grid,
                                        _stream_management, "qam",self. _num_bits_per_symbol, dtype=tf.complex64)
        self._layer_demapper = LayerDemapper(self._layer_mapper, num_bits_per_symbol=self._num_bits_per_symbol)
        self._tb_decoder = TBDecoder(self._tb_encoder, output_dtype=tf.float32)




        # Configure antenna arrays
        _ue_antenna = Antenna(polarization="single",
                        polarization_type="V",
                        antenna_pattern="38.901",
                        carrier_frequency=_carrier_frequency)

        _gnb_array = AntennaArray(num_rows=1,
                                num_cols=_num_rx_ant//2,
                                polarization="dual",
                                polarization_type="cross",
                                antenna_pattern="38.901",
                                carrier_frequency=_carrier_frequency)

        self._channel_model = CDL(model = 'A',
                                  delay_spread = 150e-9,
                                  carrier_frequency = _carrier_frequency,
                                  ut_array = _ue_antenna,
                                  bs_array = _gnb_array,
                                  direction = _link_direction,
                                  min_speed = 0)

        # self._channel_model = UMi(carrier_frequency=_carrier_frequency,
        #                           o2i_model="low",
        #                           ut_array=_ue_antenna,
        #                           bs_array=_gnb_array,
        #                           direction="uplink",
        #                           enable_pathloss=False,
        #                           enable_shadow_fading=False)

        self._channel = OFDMChannel(
                            self._channel_model,
                            self._resource_grid,
                            normalize_channel=True,
                            return_channel=True)



    def new_topology(self, batch_size):
        """Set new topology"""
        topology = gen_topology(batch_size,
                                self._num_tx,
                                'umi',
                                min_ut_velocity=0,
                                max_ut_velocity=0)

        self._channel_model.set_topology(*topology)

    @tf.function(jit_compile=True)
    def call(self, batch_size, ebno_db):
        # self.new_topology(batch_size)



        b = self._binary_source([batch_size, self._num_tx, self._tb_size])
        c = self._tb_encoder(b)
        x_map = self._mapper(c)
        x_layer = self._layer_mapper(x_map)
        x = self._resource_grid_mapper(x_layer)


        no = ebnodb2no(ebno_db,
                       self._num_bits_per_symbol,
                       self._target_coderate,
                       self._resource_grid)
        y, h = self._channel([x, no])


        h_hat,err_var = self._channel_estimator([y, no])
        # h_hat = h[...,410:-410]

        llr_det = self._mimo_detector([y, h_hat, err_var, no])



        llr = self._model(inputs)

        llr = tf.concat([llr[...,0:2,:],llr[...,3:11,:], llr[...,12:14,:]],axis=-2)
        llr = tf.transpose(llr, [0, 2, 1, 3])
        llr_det = tf.reshape(llr, [tf.shape(llr)[0], 1, 1, -1])




        llr_layer = self._layer_demapper(llr_det)

        print(no, llr_layer)
        bce = tf.nn.sigmoid_cross_entropy_with_logits(c, llr_layer)
        bce = tf.reduce_mean(bce)
        print(bce)


        b_hat, tb_crc_status = self._tb_decoder(llr_layer)

        return b, b_hat

XLA can lead to reduced numerical precision. Use with care.


In [ ]:
b, b_hat = Model()(1,7.0)
compute_ber(b, b_hat)

tf.Tensor(0.50821376, shape=(), dtype=float32) tf.Tensor(
[[[ 0.39727205 -0.06281994 -0.63921374 ...  0.00379648 -0.2373026
   -0.23420796]]], shape=(1, 1, 78624), dtype=float32)
tf.Tensor(0.75021863, shape=(), dtype=float32)


<tf.Tensor: shape=(), dtype=float64, numpy=0.49917796958487465>

In [ ]:
PUSCH_SIMS = {
    "bler" : [],
    "ber" : []
    }

start = time.time()

e2e = Model()
ebno_dbs = np.arange(-5., # Min SNR for evaluation
                     10., # Max SNR for evaluation
                     0.5)

ber, bler = sim_ber(e2e, ebno_dbs, batch_size=16, num_target_block_errors=128, max_mc_iter=100)
print(ber, bler)

PUSCH_SIMS["ber"].append(list(ber.numpy()))
PUSCH_SIMS["bler"].append(list(bler.numpy()))

PUSCH_SIMS["duration"] = time.time() - start

EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
     -5.0 | 2.9842e-01 | 1.0000e+00 |      743469 |     2491392 |          128 |         128 |       234.2 |reached target block errors
     -4.5 | 2.3674e-01 | 1.0000e+00 |      589803 |     2491392 |          128 |         128 |        72.3 |reached target block errors
     -4.0 | 1.5634e-01 | 1.0000e+00 |      389504 |     2491392 |          128 |         128 |        72.0 |reached target block errors
     -3.5 | 1.1151e-01 | 1.0000e+00 |      277803 |     2491392 |          128 |         128 |        72.0 |reached target block errors
     -3.0 | 8.0396e-02 | 9.7917e-01 |      225335 |     2802816 |          141 |         144 |        89.3 |reached target block errors


KeyboardInterrupt: 